In [1]:
from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
import dask.array as da
from glob import glob

filenames = sorted(glob(r"D:\Work\lab\Data\JUNE2022\a473_baselinez_res_frame_roundtrip.tif.frames\*.tif"), key=alphanumeric_key)
# read the first file to get the shape and dtype
# ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE
sample = imread(filenames[0])

lazy_imread = delayed(imread)  # lazy reader
lazy_arrays = [lazy_imread(fn) for fn in filenames]
dask_arrays = [
    da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
    for delayed_reader in lazy_arrays
]
# Stack into one large dask.array
stack = da.stack(dask_arrays, axis=0)
stack.shape  # (nfiles, nz, ny, nx)

# in jupyter notebook the repr of a dask stack provides a useful visual:
stack

dask.array<stack, shape=(51, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>

In [ ]:
import napari
from dask_image.imread import imread
import numpy as np
from scipy import ndimage as ndi

import napari

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

stack = imread(r"D:\Work\lab\Data\JUNE2022\a473_baselinez_res_frame_roundtrip.tif.frames\*.tif")
viewer = napari.view_image(
    stack,
    contrast_limits=[0, 2000],
    multiscale=False
)

In [ ]:
from skimage import filters


edges = filters.scharr(stack)

nuclei_layer = viewer.layers['stack']
nuclei_layer.blending = 'additive'
nuclei_layer.colormap = 'green'

viewer.add_image(
    edges,
    scale=spacing,
    blending='additive',
    colormap='magenta',
)
edges.shape
viewer.layers['stack'].visible = False

In [ ]:
li_thresholded = denoised > filters.threshold_li(denoised) + 0.01
viewer.add_image(
    li_thresholded,
    opacity=0.3,
)
viewer.layers['edges'].visible = False